In [1]:
## Github Link - https://github.com/skeetercathcart/CS445_FinalProject_F24

import csv
import pandas as pd
import string
from sympy import isprime, mod_inverse
from itertools import permutations
import timeit
import time
import math

#df = pd.read_csv('CS445_Primes.csv')

# Sieve of Eratosthenes, used to generate prime numbers sequentially. Used to generate a list of prime numbers
def sieve_of_eratosthenes(limit):
    primes = []
    is_prime = [True] * (limit + 1)
    is_prime[0] = is_prime[1] = False 
    
    for number in range(2, limit + 1):
        if is_prime[number]:
            primes.append(number)
            for multiple in range(number * number, limit + 1, number):
                is_prime[multiple] = False
    
    return primes


# Used to write primes to .csv in hopes of saving time when brute forcing decryption
def write_primes_to_csv(primes, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Prime Number'])  # Write the header
        for prime in primes:
            writer.writerow([prime])  # Write each prime number in a new row



In [2]:
# Function for generating a valid e and d value for the encryption
def check_e_d(p, q):
    # Temp vars because d may be used throughout this notebook
    d = 0
    coprime = False

    # Go through the list of small primes to check if there is a valid value of d for the given p and q. If not, go to the next prime value
    # for e and see if there is a valid d
    for element in first_primes_list[3:]:
        # We use a try-except here because the pow function will throw a ValueError, so we catch with an 'except' rather than using 'if'
        try: 
            totient = math.lcm(p - 1, q - 1)
            d = pow(element, -1, totient)
            # If d is an integer, we then check if it is coprime to our potential e value. If e is valid, we can return e and d
            if(d != 0 and totient % element != 0):
                return [element, d, totient, True]      
        # If we get an error back, rather than value for d, loop again with the next prime value from the list
        except Exception as ex:
            continue
        # If the whole list of first primes is exhausted, return 0s
        else:
            return [0, 0, 0, False]


# Putting it all together 
def generate_RSA(bit_size, message):
    
    # Assign k value
    k = 3

    # Generate Initial Primes Numbers
    coprime = False
    P = 0
    Q = 1
    while(coprime == False and P != Q):

        RSA_Primes = generate_primes(bit_size)
        [P, Q] = RSA_Primes

        # Calculate n
        n = P * Q

        # Calculate d (modular multiplicative inverse) and e (small prime) that are coprime
        [e, d, totient, coprime] = check_e_d(P, Q)

    # Encrypt a message
    cipher_int = encrypt_message(message)
    print('New ' + str(bit_size) + ' bit RSA generated with [p, q, n, e, enrypted text] = [' + str(P) + ', ' + str(Q) + ', ' + str(n) + ', ' + str(e) + ', ' + str(cipher_int) + ']')
    return [n, e, cipher_int]


In [3]:
## Github Link - https://github.com/skeetercathcart/CS445_FinalProject_F24

import random

# Pre generated primes
first_primes_list = [3, 5, 7, 11, 13, 17, 19, 23, 29,
                     31, 37, 41, 43, 47, 53, 59, 61, 67,
                     71, 73, 79, 83, 89, 97, 101, 103,
                     107, 109, 113, 127, 131, 137, 139,
                     149, 151, 157, 163, 167, 173, 179,
                     181, 191, 193, 197, 199, 211, 223,
                     227, 229, 233, 239, 241, 251, 257,
                     263, 269, 271, 277, 281, 283, 293,
                     307, 311, 313, 317, 331, 337, 347, 349]


# Functions for generating random primes

# Random number generator that gives a number that is of size n bits
def nBitRandom(n):
    return random.randrange(2**(n-1)+1, 2**n - 1)
 

def getLowLevelPrime(n):
    #Generate a prime candidate divisible by first primes
    while True:
        # Obtain a random number
        pc = nBitRandom(n)
 
        # Test divisibility by pre-generated primes
        for divisor in first_primes_list:
            if pc % divisor == 0 and divisor**2 <= pc:
                break
        else:
            return pc

# Primality test function
def isMillerRabinPassed(mrc):
    # Run 20 iterations of Rabin Miller Primality test
    maxDivisionsByTwo = 0
    ec = mrc-1
    while ec % 2 == 0:
        ec >>= 1
        maxDivisionsByTwo += 1
    assert(2**maxDivisionsByTwo * ec == mrc-1)
 
    def trialComposite(round_tester):
        if pow(round_tester, ec, mrc) == 1:
            return False
        for i in range(maxDivisionsByTwo):
            if pow(round_tester, 2**i * ec, mrc) == mrc-1:
                return False
        return True
 
    # Set number of trials here
    numberOfRabinTrials = 20
    for i in range(numberOfRabinTrials):
        round_tester = random.randrange(2, mrc)
        if trialComposite(round_tester):
            return False
    return True

# Function that combines above functions to generate two prime numbers of a given number of bits
def generate_primes(bit_size):
    RSA_Primes = []
    while True:
        n = bit_size
        P = getLowLevelPrime(n)
        if not isMillerRabinPassed(P):
            continue
        else:
            break
    while True:
        n = bit_size
        Q = getLowLevelPrime(n)
        if not isMillerRabinPassed(Q):
            continue
        else:
            break
    RSA_Primes.append(P)
    RSA_Primes.append(Q)
    return(RSA_Primes)

def generate_public_key(message_int, e, n):
    return pow(encoded_int, e) % n

def generate_private_key(public_key, d, n):
    return pow(public_key, d) % n

# Function to send original message string and returns an integer base on utf-8 conversion
def encrypt_message(message):

    # Encode the string to bytes
    encoded_bytes = message.encode('utf-8')

    # Convert bytes to an integer
    encoded_int = int.from_bytes(encoded_bytes, byteorder='big')

    return encoded_int

# Function that takes private information (d) and returns message string
def decrypt_message(public_key, d, n):
    # Decode public key int to bytes
    decoded_bytes = public_key.to_bytes((public_key.bit_length() + 7) // 8, byteorder='big')

    # Decode the bytes back to a string
    decoded_string = decoded_bytes.decode('utf-8')

    return decoded_string

In [4]:
def brute_force_rsa(brute_n, brute_e, encoded_int, bit_size):
    
    count = 0

    # Use sieve to generate a list of all possible primes up to the highest prime of the given bit size used for RSA
    primes = sieve_of_eratosthenes(pow(2, bit_size))

    # Timestamp
    sieve_done = time.time()
    
    # Loop through potential prime factors
    # Enumerate is used here to save time because order doesn't matter in combinations of p and q (i.e. p = 3 & q = 5 is the same as p = 5 & q = 3)
    for i, p in enumerate(primes):  # Assuming your DataFrame has a column named 'Primes'
        for q in primes[i+1:]:

            # A counter for my personal interest, to see progress of the loop. Not needed for functionality
            count += 1
            if count % 1000000 == 0:
                print('Iteration: ' + str(count) + ' | ' + str(p) + ', ' + str(q))

            # Double check p and q are prime and ensure that p * q matches the public n value
            if isprime(p) and isprime(q) and p * q == brute_n:

                # Timestamp
                keys_found = time.time()

                # Quick log to verify the p and q values actually match what was generated by generate_RSA function
                print('found keys p & q: [' + str(p) + ', ' + str(q) + ']')

                
                try:
                    # Calculate the decryption key d
                    phi_n = (p - 1) * (q - 1)  # Euler's totient
                    d = pow(brute_e, -1, phi_n)  # Modular inverse 
                    
                    # Decrypt the message
                    plaintext = decrypt_message(encoded_int, d, brute_n)

                    # Timestamp
                    decrypt_done = time.time()
                    result = [sieve_done, keys_found, decrypt_done]
                    print('Found keys with p = ' + str(p) + ' and q = ' + str(q) + '. Decrypted message: ' + str(plaintext))
                    return result
                except ValueError as e:
                    continue 

    return 'No valid prime factors found.'

In [5]:
# Set values for bit sizes to test in loop below
test_bit_sizes = [8, 16]

# Create dataframe to store timestamps
column_names = ['Bit Size', 'Start Time', 'Sieve Done', 'Keys Found', 'Decrypt Done']
result_df = pd.DataFrame(columns = column_names)

In [ ]:
# Test 
for size in test_bit_sizes:
    start_time  = time.time()
    RSA = generate_RSA(size, 'hello world')
    times = brute_force_rsa(RSA[0], RSA[1], RSA[2], size)
    result_df.loc[len(result_df)] = [size, start_time, times[0], times[1], times[2]]

New 8 bit RSA generated with [p, q, n, e, enrypted text] = [223, 233, 51959, 11, 126207244316550804821666916]
found keys p & q: [223, 233]
Found keys with p = 223 and q = 233. Decrypted message: hello world
New 16 bit RSA generated with [p, q, n, e, enrypted text] = [53323, 44111, 2352130853, 13, 126207244316550804821666916]
Iteration: 1000000 | 907, 44483
Iteration: 2000000 | 2083, 15493
Iteration: 3000000 | 3373, 60169
Iteration: 4000000 | 4789, 9587


In [ ]:
result_df